In [1]:
import json
from web3 import Web3
import time



In [2]:
# Connect to an Ethereum node (replace with your own node URL or Infura endpoint)
web3_provider = 'http://localhost:7545'
w3 = Web3(Web3.HTTPProvider(web3_provider))

In [3]:
if w3.is_connected():
    print("Successfully connected to the node at", web3_provider)
else:
    print("Connection to node failed. Please check the URL and try again.")


Successfully connected to the node at http://localhost:7545


In [4]:
# Your Ethereum account private key (NEVER share or commit this!)
private_key = '0xb99052926180cf5daff5b20633add14f33655b4711ede3c3d4ddb40751dc4311'



In [5]:
#collection json
with open(r'..\truffle_compiled_contract\build\contracts\CollectionContract.json') as f:
    collection_json = json.load(f)

with open(r'..\truffle_compiled_contract\build\contracts\NFTContract.json') as f:
    nft_json = json.load(f)


with open(r'..\truffle_compiled_contract\build\contracts\NFTAccessControl.json') as f:
    NFTAccessControl_json = json.load(f)


with open(r'..\truffle_compiled_contract\build\contracts\NFTMetadata.json') as f:
    NFTMetadata_json = json.load(f)

In [6]:
with open(r'..\truffle_compiled_contract\build\contractAddresses\development_addresses.json') as f:
    address_json= json.load(f)

In [7]:
address_json

{'MasterAccessControl': '0x2c0ecae1970c2Ed82a52285F1e78f540Cb106527',
 'NFTAccessControl': '0x3EF03Fda5c2E36b6b2986D1af812bb5f3b6Ef1a0',
 'NFTMetadata': '0x01A822E80Bf2A701eb8e62225e97B58111c24D94',
 'NFTContract': '0xACe85a52Ae338436685c31F72cAAB1f691d6C9DC',
 'CollectionContract': '0x3019B696ae02356a8B545EB0bcFcE6dd1255Ac10'}

In [8]:
def write_to_contract(contract, function_name: str, 
                      arguments: list, private_key: str, 
                      web3_provider: str,
                      gas = 2000000):
    """
    Write data to a contract function
    :param contract: The contract object
    :param function_name: The name of the function to call
    :param arguments: The arguments to pass to the function
    :param private_key: The private key of the account calling the function
    :param web3_provider: The URL of the Ethereum node
    :return: The transaction receipt
    """
    # Connect to the network
    w3 = Web3(Web3.HTTPProvider(web3_provider))

    # Checking if the connection to the node was successful
    if w3.is_connected():
        print("Successfully connected to the node at", web3_provider)
    else:
        print("Connection to node failed. Please check the URL and try again.")

    account = w3.eth.account.from_key(private_key)

    # Write data to the function
    nonce = w3.eth.get_transaction_count(account.address)
    transaction = contract.functions[function_name](*arguments).build_transaction({
        "from": account.address,
        "nonce": nonce,
        "gas": gas,
        "gasPrice": w3.eth.gas_price
    })
    signed_txn = account.sign_transaction(transaction)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

    return tx_receipt



# Collection

In [9]:
# Your Ethereum address
account = w3.eth.account.from_key(private_key)

collection_contract = w3.eth.contract(address=address_json["CollectionContract"], abi=collection_json["abi"])


def create_collection(collection):
    

        
    # write to contract
    
    
    function_name = "createCollection"
    arguments = [collection['name'], collection['contextWindow'], collection['model'], collection['image'], collection['description']]
    key = private_key
    
    
    try :
        tx_receipt = write_to_contract(collection_contract, function_name, arguments, key, web3_provider)
        create_event = collection_contract.events.CollectionCreated().process_receipt(tx_receipt)[0]
        collection_id = create_event['args']['collectionId']
        return collection_id
    except:
        print("Error creating collection : ", collection['name'])

def getAllCollections():
    
    try:
        # Get the total number of collections
        total_collections = collection_contract.functions.getTotalCollections().call()

        all_collections = []
        for i in range(1, total_collections + 1):  # Assuming collection IDs start from 1
            collection_data = collection_contract.functions.getCollectionMetadata(i).call()
            
            # Format the collection data
            formatted_collection = {
                "id": i,
                "name": collection_data[0],
                "contextWindow": collection_data[1],
                "model": collection_data[2],
                "image": collection_data[3],
                "description": collection_data[4],
                "creator": collection_data[5],
                "date": collection_data[6],
                "owner": collection_data[7],
                "collectionaddress": f"#{i}",
                "noOfNFTs": collection_contract.functions.getCollectionNFTCount(i).call()
            }
            all_collections.append(formatted_collection)

        return all_collections
    except Exception as e:
        print(f"Error getting all collections: {e}")
        return None


def getAllCollections_by_address(address):
    
    try:
        all_collections = getAllCollections()
        collections = []
        
        for collection in all_collections:
            if collection['owner'] == address:
                collections.append(collection)
    except Exception as e:
        print(f"Error getting all collections: {e}")
        return None
    
    return collections


def get_collection_details_by_id(collection_id):
    try:
        # Call the getCollectionMetadata function
        metadata = collection_contract.functions.getCollectionMetadata(collection_id).call()
        
        # Get additional details
        nft_count = collection_contract.functions.getCollectionNFTCount(collection_id).call()
        owner = collection_contract.functions.getCollectionOwner(collection_id).call()
        unique_holders = collection_contract.functions.getCollectionUniqueHolders(collection_id).call()

        # Format the collection data
        collection_details = {
            "id": collection_id,
            "name": metadata[0],
            "contextWindow": metadata[1],
            "baseModel": metadata[2],
            "image": metadata[3],
            "description": metadata[4],
            "creator": metadata[5],
            "dateCreated": metadata[6],
            "owner": owner,
            "collectionaddress": f"#{collection_id}",
            "noOfNFTs": nft_count,
            "uniqueHolders": unique_holders
        }

        return collection_details
    except Exception as e:
        print(f"Error getting details for collection ID {collection_id}: {e}")
        return None


In [10]:
with open('./data/all_collections.json') as f:
    collections = json.load(f)

In [11]:
for collection in collections["collections"]:
    collection_id = create_collection(collection)
    print(f"Collection ID: {collection_id}")

Successfully connected to the node at http://localhost:7545
Collection ID: 1
Successfully connected to the node at http://localhost:7545
Collection ID: 2
Successfully connected to the node at http://localhost:7545
Collection ID: 3
Successfully connected to the node at http://localhost:7545
Collection ID: 4
Successfully connected to the node at http://localhost:7545
Collection ID: 5
Successfully connected to the node at http://localhost:7545
Collection ID: 6
Successfully connected to the node at http://localhost:7545
Collection ID: 7
Successfully connected to the node at http://localhost:7545
Collection ID: 8
Successfully connected to the node at http://localhost:7545
Collection ID: 9
Successfully connected to the node at http://localhost:7545
Collection ID: 10
Successfully connected to the node at http://localhost:7545
Collection ID: 11
Successfully connected to the node at http://localhost:7545
Collection ID: 12
Successfully connected to the node at http://localhost:7545
Collection ID

In [12]:
getAllCollections()

[{'id': 1,
  'name': 'Neural Mint Hub',
  'contextWindow': 4096,
  'model': 'Llama 3.1 70B',
  'image': 'http://localhost:5500/image/temp7.png',
  'description': 'Discover the fusion of neural networks and blockchain at the Neural Mint Hub. This collection offers unique AI-generated NFTs crafted by advanced neural models.',
  'creator': '0x53E5dA1dC6D0C6ef7e85ed52C2A256dd1AA669bd',
  'date': 1728650223,
  'owner': '0x53E5dA1dC6D0C6ef7e85ed52C2A256dd1AA669bd',
  'collectionaddress': '#1',
  'noOfNFTs': 0},
 {'id': 2,
  'name': 'Quantum Rewards',
  'contextWindow': 2048,
  'model': 'Llama 3.1',
  'image': 'http://localhost:5500/image/temp2.png',
  'description': 'Unlock quantum leaps in rewards with our AI-enhanced NFT collection. Each piece represents the convergence of quantum computing concepts and digital art.',
  'creator': '0x53E5dA1dC6D0C6ef7e85ed52C2A256dd1AA669bd',
  'date': 1728650223,
  'owner': '0x53E5dA1dC6D0C6ef7e85ed52C2A256dd1AA669bd',
  'collectionaddress': '#2',
  'noOf

In [13]:
get_collection_details_by_id(1)

{'id': 1,
 'name': 'Neural Mint Hub',
 'contextWindow': 4096,
 'baseModel': 'Llama 3.1 70B',
 'image': 'http://localhost:5500/image/temp7.png',
 'description': 'Discover the fusion of neural networks and blockchain at the Neural Mint Hub. This collection offers unique AI-generated NFTs crafted by advanced neural models.',
 'creator': '0x53E5dA1dC6D0C6ef7e85ed52C2A256dd1AA669bd',
 'dateCreated': 1728650223,
 'owner': '0x53E5dA1dC6D0C6ef7e85ed52C2A256dd1AA669bd',
 'collectionaddress': '#1',
 'noOfNFTs': 0,
 'uniqueHolders': 0}

# NFT Cretion

In [34]:
# Your Ethereum address
account = w3.eth.account.from_key(private_key)

# collection_contract = w3.eth.contract(address=address_json["CollectionContract"], abi=collection_json["abi"])
nft_contract = w3.eth.contract(address=address_json["NFTContract"], abi=nft_json["abi"])
nft_metadata_contract = w3.eth.contract(address=address_json["NFTMetadata"], abi=NFTMetadata_json["abi"])
nft_access_control_contract = w3.eth.contract(address=address_json["NFTAccessControl"], abi=NFTAccessControl_json["abi"])


def create_NFT(nft_data):
    

        
    # write to contract
    
    
    function_name = "createNFT"
    
    _collectionId = nft_data['collection_id']
    _name = nft_data['name']
    level_of_ownership = 6
    
    
    arguments = [_collectionId, _name, level_of_ownership]
    key = private_key
    
    
    try :
        tx_receipt = write_to_contract(nft_contract, function_name, arguments, key, web3_provider)
        create_event = nft_contract.events.NFTCreated().process_receipt(tx_receipt)[0]
        collection_id = create_event['args']['collectionId']
        nft_id = create_event['args']['tokenId']
        cretor = create_event['args']['creator']
        
        print (f"nft {nft_id} created by {cretor} in collection {collection_id}")
        
        return nft_id
    except:
        print("Error creating nft : ", collection['name'])
        

def create_metadata(nft_data):
    

        
    # write to contract
    
    
    function_name = "createMetadata"
    
    _collectionId = nft_data['collection_id']
    _nftId = nft_data['id']
    # _metadata = {
    #     "image" : nft_data['image'],
    #     "model" : nft_data['model'],
    #     "data"  : "http//localhost:5500/data",
    #     "rag"   : "http//localhost:5500/rag",
    #     "fineTuneData" : "localhost:5500/fineTuneData",
    #     "description" : nft_data['description']
        
    # }
    
    
    _metadata = [
        nft_data['image'],
        nft_data['model'],
        "http//localhost:5500/data",
        "http//localhost:5500/rag",
        "localhost:5500/fineTuneData",
        nft_data['description']
    ]
    
    
    
    arguments = [_collectionId, _nftId, _metadata]
    key = private_key
    
    
    try :
        tx_receipt = write_to_contract(nft_metadata_contract, function_name, arguments, key, web3_provider)
        create_event = nft_metadata_contract.events.MetadataCreated().process_receipt(tx_receipt)[0]
        collection_id = create_event['args']['collectionId']
        nft_id = create_event['args']['nftId']
        metadata = create_event['args']['metadata']
        
        # print (f"nft {nft_id} created by {cretor} in collection {collection_id}")
        
        print (f"metadata for nft {nft_id} created in collection {collection_id}")
        
        return nft_id
    except:
        print("Error creating nft metadata: ", nft_data['name'])
        
def provide_access(nft_data):
    
    
    if "accessList" not in nft_data:
        return 0
    
    addresses =[
        "0xcb6d7cDCa0563575b6b734FA4f3e9d6ac7542912",
        "0xC1BA95aE30C40Bde9259B23E9706B359119af8a8",
        "0x896b05e46Fa4Ec0CCBCff4FD70E040B24B705b13",
        "0xA6BBfa08C6Bec195FB4138A3225B7b6df9aA75eb",
        "0x51414DFE8A683aA64bE0273C6819eB410939395B"
    ]
    
    _access_list_temp = []
    count_temp = len(addresses)
    
    for i in nft_data['accessList']:
        if count_temp == 0:
            break
        _access_list_temp.append({ "address":addresses[len(addresses)-count_temp],  "accessLevel":i["accessLevel"]})
        count_temp = count_temp - 1
        
    _access_list = _access_list_temp
    
    
    function_name = "grantAccess"
    
    _collectionId = nft_data['collection_id']
    _nftId = nft_data['id']
    
    key = private_key
    
    
    print("Providing access for nft : ", nft_data['name'])
    
    for access in _access_list:
        
        _user = access['address']
        _level = access['accessLevel']
        
        arguments = [_collectionId, _nftId, _user, _level]
        
        try :
            
            
            tx_receipt = write_to_contract(nft_access_control_contract, function_name, arguments, key, web3_provider)
            create_event = nft_access_control_contract.events.AccessGranted().process_receipt(tx_receipt)[0]
            collection_id = create_event['args']['collectionId']
            nft_id = create_event['args']['nftId']
            user = create_event['args']['user']
            accesslevel = create_event['args']['accessLevel']
            
            
            print("\t\t Access granted to user : ", user , " with access level : ", accesslevel)
            
            # print (f"nft {nft_id} created by {cretor} in collection {collection_id}") 
            # return nft_id
        except:
            print("\t\t Error granding user : ", _user) 
        
        
        
        
    
        


In [36]:
with open('./data/all_nft_tron.json') as f:
    all_nft_tron = json.load(f)

In [20]:
all_nft_tron

{'nfts': [{'id': 1,
   'name': 'Cognitive Llama',
   'collection': 'Neural Mint Hub',
   'collection_id': 1,
   'image': 'http://localhost:5500/image/temp11.jpg',
   'model': 'Llama 3.1',
   'attributes': [{'trait_type': 'Model', 'value': 'Llama 3.1'},
    {'trait_type': 'Context Window', 'value': '4096 tokens'},
    {'trait_type': 'Total Access', 'value': '1'},
    {'trait_type': 'Collection', 'value': 'Neural Mint Hub'}],
   'accessList': [{'address': 'TLTtQz2Njap4nPAQGPGJn5EEksoM1uw4Qr',
     'accessLevel': 'Level 6'},
    {'address': 'TFJGXg1C962aDqSyvdSZXZ9am2ETqwNQka',
     'accessLevel': 'Level 5'},
    {'address': 'TThwHjyzx2E1TNN4GdSoaHeCRJVi3EuKS7',
     'accessLevel': 'Level 2'},
    {'address': 'TJfqnAUwX2ViGTP1J81itXWhuJhUYa8RwK',
     'accessLevel': 'Level 2'},
    {'address': 'TVh65XCYkerBch3WtgY9kwLqrtH64MhiFe',
     'accessLevel': 'Level 3'},
    {'address': 'TMVQGm1qAQYVdetCeGRRkTWYYrLXuHK2HC',
     'accessLevel': 'Level 1'}],
   'tokenId': '1234',
   'tokenStandard':

In [21]:
for nft in all_nft_tron["nfts"]:
    nft_id = create_NFT(nft)
    nft['id'] = nft_id
    print(f"NFT ID: {nft_id}")

Successfully connected to the node at http://localhost:7545
nft 1 created by 0x53E5dA1dC6D0C6ef7e85ed52C2A256dd1AA669bd in collection 1
NFT ID: 1
Successfully connected to the node at http://localhost:7545
nft 2 created by 0x53E5dA1dC6D0C6ef7e85ed52C2A256dd1AA669bd in collection 1
NFT ID: 2
Successfully connected to the node at http://localhost:7545
nft 3 created by 0x53E5dA1dC6D0C6ef7e85ed52C2A256dd1AA669bd in collection 1
NFT ID: 3
Successfully connected to the node at http://localhost:7545
nft 1 created by 0x53E5dA1dC6D0C6ef7e85ed52C2A256dd1AA669bd in collection 2
NFT ID: 1
Successfully connected to the node at http://localhost:7545
nft 2 created by 0x53E5dA1dC6D0C6ef7e85ed52C2A256dd1AA669bd in collection 2
NFT ID: 2
Successfully connected to the node at http://localhost:7545


f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0xddc2f090e6b63ea39d3966226509586cffe0d4a9cb0d45d2adc07e11e5138c74') and logIndex: 0 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0xddc2f090e6b63ea39d3966226509586cffe0d4a9cb0d45d2adc07e11e5138c74') and logIndex: 1 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0xddc2f090e6b63ea39d3966226509586cffe0d4a9cb0d45d2adc07e11e5138c74') and logIndex:

nft 4 created by 0x53E5dA1dC6D0C6ef7e85ed52C2A256dd1AA669bd in collection 1
NFT ID: 4
Successfully connected to the node at http://localhost:7545
nft 5 created by 0x53E5dA1dC6D0C6ef7e85ed52C2A256dd1AA669bd in collection 1
NFT ID: 5
Successfully connected to the node at http://localhost:7545
nft 6 created by 0x53E5dA1dC6D0C6ef7e85ed52C2A256dd1AA669bd in collection 1
NFT ID: 6
Successfully connected to the node at http://localhost:7545
nft 7 created by 0x53E5dA1dC6D0C6ef7e85ed52C2A256dd1AA669bd in collection 1
NFT ID: 7


f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0xb0683204954bf2aeffa7cd5994f13fb353731862f60ba3b7847ca319da569d6e') and logIndex: 0 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0xb0683204954bf2aeffa7cd5994f13fb353731862f60ba3b7847ca319da569d6e') and logIndex: 1 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0xb0683204954bf2aeffa7cd5994f13fb353731862f60ba3b7847ca319da569d6e') and logIndex:

In [33]:
for nft in all_nft_tron["nfts"]:
    nft_id = create_metadata(nft)
    # nft['id'] = nft_id
    # print(f"NFT ID: {nft_id}")

Successfully connected to the node at http://localhost:7545
metadata for nft 1 created in collection 1
Successfully connected to the node at http://localhost:7545
metadata for nft 2 created in collection 1
Successfully connected to the node at http://localhost:7545
metadata for nft 3 created in collection 1
Successfully connected to the node at http://localhost:7545
metadata for nft 1 created in collection 2
Successfully connected to the node at http://localhost:7545
metadata for nft 2 created in collection 2
Successfully connected to the node at http://localhost:7545
metadata for nft 4 created in collection 1
Successfully connected to the node at http://localhost:7545
metadata for nft 5 created in collection 1
Successfully connected to the node at http://localhost:7545
metadata for nft 6 created in collection 1
Successfully connected to the node at http://localhost:7545
metadata for nft 7 created in collection 1


In [37]:
for nft in all_nft_tron["nfts"]:
    nft_id = provide_access(nft)
    # nft['id'] = nft_id
    # print(f"NFT ID: {nft_id}")

Providing access for nft :  Cognitive Llama
Successfully connected to the node at http://localhost:7545
		 Access granted to user :  0xcb6d7cDCa0563575b6b734FA4f3e9d6ac7542912  with access level :  6
Successfully connected to the node at http://localhost:7545
		 Access granted to user :  0xC1BA95aE30C40Bde9259B23E9706B359119af8a8  with access level :  5
Successfully connected to the node at http://localhost:7545
		 Access granted to user :  0x896b05e46Fa4Ec0CCBCff4FD70E040B24B705b13  with access level :  2
Successfully connected to the node at http://localhost:7545
		 Access granted to user :  0xA6BBfa08C6Bec195FB4138A3225B7b6df9aA75eb  with access level :  2
Successfully connected to the node at http://localhost:7545
		 Access granted to user :  0x51414DFE8A683aA64bE0273C6819eB410939395B  with access level :  3
Providing access for nft :  Cyborg Bored Ape
Successfully connected to the node at http://localhost:7545
		 Access granted to user :  0xcb6d7cDCa0563575b6b734FA4f3e9d6ac7542912

f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0x6ff3aa4a6c7e0d23ba0a3102b84212e0c98ee4228cc69436d2aa86ad765e645a') and logIndex: 1 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0xdfa524fa413e3a23f0c378df5d93af01a53d221be5e6265ffa0f6974c7cee771') and logIndex: 1 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0xbe0ebd1bc838514a9e59b3df83383a52d8fe7046a2bcd6f6dbf9448dde04806c') and logIndex:

		 Access granted to user :  0xC1BA95aE30C40Bde9259B23E9706B359119af8a8  with access level :  2
Providing access for nft :  MankiBean
Successfully connected to the node at http://localhost:7545
		 Access granted to user :  0xcb6d7cDCa0563575b6b734FA4f3e9d6ac7542912  with access level :  1
Providing access for nft :  based punk
Successfully connected to the node at http://localhost:7545
		 Access granted to user :  0xcb6d7cDCa0563575b6b734FA4f3e9d6ac7542912  with access level :  6
Successfully connected to the node at http://localhost:7545
		 Access granted to user :  0xC1BA95aE30C40Bde9259B23E9706B359119af8a8  with access level :  2
Providing access for nft :  Dark Llama
Successfully connected to the node at http://localhost:7545
		 Access granted to user :  0xcb6d7cDCa0563575b6b734FA4f3e9d6ac7542912  with access level :  1
Providing access for nft :  Tron Cells
Successfully connected to the node at http://localhost:7545
		 Access granted to user :  0xcb6d7cDCa0563575b6b734FA4f3e9d6ac7

f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0xb44515203aca1db0abf3e9df4d8be273ebe46275bc4a82a2efbb02686f3560c7') and logIndex: 1 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0x99f2b4eba575a54369d345e60822e7a8a83f20d5eaccf6301cf75e9e925f1397') and logIndex: 1 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(


In [40]:
def all_nft_information(nft_id, collection_id):
    try:
        nft_info = nft_contract.functions.getNFTInfo(collection_id, nft_id).call()
        metadata = nft_metadata_contract.functions.getMetadata(collection_id, nft_id).call()
        
        return {
            "id": nft_id,
            "collectionId": collection_id,
            "levelOfOwnership": nft_info[0],
            "name": nft_info[1],
            "creator": nft_info[2],
            "creationDate": nft_info[3],
            "owner": nft_info[4],
            "metadata": {
                "image": metadata[0],
                "baseModel": metadata[1],
                "data": metadata[2],
                "rag": metadata[3],
                "fineTuneData": metadata[4],
                "description": metadata[5]
            }
        }
    except Exception as e:
        print(f"Error getting information for NFT ID {nft_id} in collection {collection_id}: {e}")
        return None

def all_nft_of_a_collection(collection_id):
    try:
        nft_count = nft_contract.functions.getCollectionNFTCount(collection_id).call()
        nft_ids = nft_contract.functions.getCollectionNFTs(collection_id).call()
        
        nfts = []
        for nft_id in nft_ids:
            nft_info = all_nft_information(nft_id, collection_id)
            if nft_info:
                nfts.append(nft_info)
        
        return nfts
    except Exception as e:
        print(f"Error getting NFTs for collection {collection_id}: {e}")
        return None

def all_access_levels_of_a_collection_nft(collection_id, nft_id):
    try:
        users_access = nft_access_control_contract.functions.getAllUsersAccessForNFT(collection_id, nft_id).call()
        
        access_levels = []
        for user_access in users_access:
            access_levels.append({
                "user": user_access[0],
                "accessLevel": user_access[1]
            })
        
        return access_levels
    except Exception as e:
        print(f"Error getting access levels for NFT ID {nft_id} in collection {collection_id}: {e}")
        return None

def all_nfts_own_or_have_access_by_user(user_address):
    try:
        user_access_entries = nft_access_control_contract.functions.getAllAccessForUser(user_address).call()
        
        nfts = []
        for entry in user_access_entries:
            collection_id, nft_id, access_level = entry
            nft_info = all_nft_information(nft_id, collection_id)
            if nft_info:
                nft_info['accessLevel'] = access_level
                nfts.append(nft_info)
        
        return nfts
    except Exception as e:
        print(f"Error getting NFTs for user {user_address}: {e}")
        return None


In [41]:
all_nft_information(1, 1)

{'id': 1,
 'collectionId': 1,
 'levelOfOwnership': 6,
 'name': 'Cognitive Llama',
 'creator': '0x53E5dA1dC6D0C6ef7e85ed52C2A256dd1AA669bd',
 'creationDate': 1728669723,
 'owner': '0x53E5dA1dC6D0C6ef7e85ed52C2A256dd1AA669bd',
 'metadata': {'image': 'http://localhost:5500/image/temp11.jpg',
  'baseModel': 'Llama 3.1',
  'data': 'http//localhost:5500/data',
  'rag': 'http//localhost:5500/rag',
  'fineTuneData': 'localhost:5500/fineTuneData',
  'description': 'Cognitive Llama is an advanced AI-driven bot, utilizing the power of Llama 3.1 with multi-level reasoning and cognition. Part of the Neural Mint Hub collection.'}}

In [42]:
all_nft_of_a_collection(1)

[{'id': 1,
  'collectionId': 1,
  'levelOfOwnership': 6,
  'name': 'Cognitive Llama',
  'creator': '0x53E5dA1dC6D0C6ef7e85ed52C2A256dd1AA669bd',
  'creationDate': 1728669723,
  'owner': '0x53E5dA1dC6D0C6ef7e85ed52C2A256dd1AA669bd',
  'metadata': {'image': 'http://localhost:5500/image/temp11.jpg',
   'baseModel': 'Llama 3.1',
   'data': 'http//localhost:5500/data',
   'rag': 'http//localhost:5500/rag',
   'fineTuneData': 'localhost:5500/fineTuneData',
   'description': 'Cognitive Llama is an advanced AI-driven bot, utilizing the power of Llama 3.1 with multi-level reasoning and cognition. Part of the Neural Mint Hub collection.'}},
 {'id': 2,
  'collectionId': 1,
  'levelOfOwnership': 6,
  'name': 'Cyborg Bored Ape',
  'creator': '0x53E5dA1dC6D0C6ef7e85ed52C2A256dd1AA669bd',
  'creationDate': 1728669723,
  'owner': '0x53E5dA1dC6D0C6ef7e85ed52C2A256dd1AA669bd',
  'metadata': {'image': 'http://localhost:5500/image/temp10.jpg',
   'baseModel': 'GPT-4',
   'data': 'http//localhost:5500/data

In [43]:
all_access_levels_of_a_collection_nft(1,1)

[{'user': '0x53E5dA1dC6D0C6ef7e85ed52C2A256dd1AA669bd', 'accessLevel': 6},
 {'user': '0xcb6d7cDCa0563575b6b734FA4f3e9d6ac7542912', 'accessLevel': 6},
 {'user': '0xC1BA95aE30C40Bde9259B23E9706B359119af8a8', 'accessLevel': 5},
 {'user': '0x896b05e46Fa4Ec0CCBCff4FD70E040B24B705b13', 'accessLevel': 2},
 {'user': '0xA6BBfa08C6Bec195FB4138A3225B7b6df9aA75eb', 'accessLevel': 2},
 {'user': '0x51414DFE8A683aA64bE0273C6819eB410939395B', 'accessLevel': 3}]

In [45]:
all_nfts_own_or_have_access_by_user("0x53E5dA1dC6D0C6ef7e85ed52C2A256dd1AA669bd")


[{'id': 1,
  'collectionId': 1,
  'levelOfOwnership': 6,
  'name': 'Cognitive Llama',
  'creator': '0x53E5dA1dC6D0C6ef7e85ed52C2A256dd1AA669bd',
  'creationDate': 1728669723,
  'owner': '0x53E5dA1dC6D0C6ef7e85ed52C2A256dd1AA669bd',
  'metadata': {'image': 'http://localhost:5500/image/temp11.jpg',
   'baseModel': 'Llama 3.1',
   'data': 'http//localhost:5500/data',
   'rag': 'http//localhost:5500/rag',
   'fineTuneData': 'localhost:5500/fineTuneData',
   'description': 'Cognitive Llama is an advanced AI-driven bot, utilizing the power of Llama 3.1 with multi-level reasoning and cognition. Part of the Neural Mint Hub collection.'},
  'accessLevel': 6},
 {'id': 2,
  'collectionId': 1,
  'levelOfOwnership': 6,
  'name': 'Cyborg Bored Ape',
  'creator': '0x53E5dA1dC6D0C6ef7e85ed52C2A256dd1AA669bd',
  'creationDate': 1728669723,
  'owner': '0x53E5dA1dC6D0C6ef7e85ed52C2A256dd1AA669bd',
  'metadata': {'image': 'http://localhost:5500/image/temp10.jpg',
   'baseModel': 'GPT-4',
   'data': 'http/